## Dataset

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np  
data = pd.read_csv("data2.csv", index_col=0, header=[0,1], parse_dates=True)

## Implementation

In [2]:
import pandas as pd
from datetime import datetime
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Load the data
data = pd.read_csv("Question2.csv", index_col=0, header=[0,1], parse_dates=True)
data = data.fillna(0)  # Replace NaN values with zeros

# Extract the da, rt, and X data
da = torch.tensor(data["da"].values).float()
rt = torch.tensor(data["rt"].values).float()
X = torch.tensor(data["X"].values).float()

# Normalize the input data
X_mean = X.mean(dim=0)
X_std = X.std(dim=0)
X_normalized = (X - X_mean) / X_std

# Extract the two-day lagged data
shifted_da = da[48:].float()
shifted_rt = rt[48:].float()

# Split the data into training and validation sets
split_idx = len(shifted_da)
X_train = X_normalized[:split_idx]
X_validate = X_normalized[split_idx:]
da_train = da[:split_idx]
da_validate = da[split_idx:]
rt_train = rt[:split_idx]
rt_validate = rt[split_idx:]

# Define the PyTorch model
class TradingModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TradingModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Define the trading loss function
def trading_loss(vl, vp, vs, sp, da, rt):
    trading_strategy = (vl * torch.clamp_min(sp - da, min=0) + vs * torch.clamp_min(da - sp, min=0))
    trading_profit = torch.sum(trading_strategy * (rt - da), dim=1)
    worst_loss = torch.min(trading_profit)
    loss = -torch.mean(trading_profit)
    if worst_loss < -1000:
        loss += torch.abs(worst_loss + 1000)
    return loss

# Hyperparameters
input_size = X.shape[1]  # Number of features
hidden_size = 10  # Number of neurons in the hidden layer
output_size = 28  # 7 bid/offer volumes + 7 bid/offer prices
lr = 0.001  # Learning rate
epochs = 100  # Training epochs

# Create the model and optimizer
model = TradingModel(input_size, hidden_size, output_size)
optimizer = optim.Adam(model.parameters(), lr=lr)

# Train the model
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(X_train)
    vl, vp, vs, sp = torch.split(output, 7, dim=1)
    loss = trading_loss(vl, vp, vs, sp, da_train, rt_train)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

# Evaluate the model on validation data
with torch.no_grad():
    output = model(X_validate)
    vl, vp, vs, sp = torch.split(output, 7, dim=1)
    profit = torch.sum((vl * torch.clamp_min(sp - da_validate, min=0) + vs * torch.clamp_min(da_validate - sp, min=0)) * (rt_validate - da_validate), dim=1)
    print(f"Validation profit: {profit.mean().item()}, Worst loss: {-torch.min(profit).item()}")


Epoch 0, Loss: 649033.0625
Epoch 10, Loss: 513674.46875
Epoch 20, Loss: 391146.25
Epoch 30, Loss: 278266.3125
Epoch 40, Loss: 187527.28125
Epoch 50, Loss: 102552.921875
Epoch 60, Loss: 72176.4765625
Epoch 70, Loss: 61674.86328125
Epoch 80, Loss: 52815.03515625
Epoch 90, Loss: 48869.17578125
Validation profit: -1.214802861213684, Worst loss: 546.2427978515625
